In [ ]:
import pandas as pd
import pyspark
sc = pyspark.SparkContext('local[*]')

In [ ]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop(['Unnamed: 0'],axis=1)
postulaciones = pd.read_csv('Sources/fiuba_4_postulaciones_concat.csv').drop(['Unnamed: 0'],axis=1)

In [ ]:
postulantes = postulantes[['idpostulante','fechanacimiento','sexo']]
postulantes.head()

In [ ]:
postulaciones.drop(['fechapostulacion'],axis=1,inplace=True)
postulaciones.head()

In [ ]:
postulaciones = postulaciones.merge(postulantes, on='idpostulante',how='right')
postulaciones.head()

In [ ]:
prom_edad = postulaciones.groupby(['idaviso']).mean()
prom_edad.head()

In [ ]:
sexo_mayoritario = postulaciones.groupby(['idaviso','sexo'],as_index=False).count()
sexo_mayoritario.head()

In [ ]:
sexo_mayoritario = sexo_mayoritario.join(pd.get_dummies(sexo_mayoritario['sexo']))

In [ ]:
sexo_mayoritario['FEM'] = sexo_mayoritario.apply(lambda x: x['idpostulante'] if x['sexo']=='FEM' else 0,axis=1)
sexo_mayoritario['MASC'] = sexo_mayoritario.apply(lambda x: x['idpostulante'] if x['sexo']=='MASC' else 0,axis=1)
sexo_mayoritario['NO_DECLARA'] = sexo_mayoritario.apply(lambda x: x['idpostulante'] if x['sexo']=='NO_DECLARA' else 0,axis=1)

In [ ]:
sexo_mayoritario = sexo_mayoritario[['idaviso','FEM','MASC','NO_DECLARA']]
sexo_mayoritario = sexo_mayoritario.groupby(['idaviso'])['FEM','MASC','NO_DECLARA'].sum()
sexo_mayoritario.head()

In [ ]:
def mayor(F,M,N):
    suma = F+M+N
    if (F/suma > 1.3 * M/suma):
        return 'FEM'
    if (M/suma > 1.3 * F/suma):
        return 'MASC'
    return 'NULL'

In [ ]:
sexo_mayoritario['TendenciaSexo'] = sexo_mayoritario.apply(lambda x: mayor(x['FEM'],x['MASC'],x['NO_DECLARA']), axis=1)

In [ ]:
sexo_mayoritario.head()

In [ ]:
sexo_mayoritario.reset_index(inplace=True)

In [ ]:
sexo_mayoritario.head()

In [ ]:
prom_edad.reset_index(inplace=True)
prom_edad.head()

In [ ]:
sexo_mayoritario = sexo_mayoritario.merge(prom_edad,on='idaviso',how='outer')
sexo_mayoritario.head()

In [ ]:
sexo_mayoritario = sexo_mayoritario[['idaviso','FEM','MASC','NO_DECLARA','TendenciaSexo','fechanacimiento']]
sexo_mayoritario.rename(columns={'fechanacimiento':'prom_edad'},inplace=True)
sexo_mayoritario.head()

In [ ]:
sexo_mayoritario.to_csv('Sources/mas_features.csv')